# 📈 Portfolio Optimization with Modern Portfolio Theory (MPT)

Welcome! In this project, we’ll build an optimized stock portfolio using **Modern Portfolio Theory**.

This notebook is designed for beginners, with step-by-step explanations.

## 🎯 Objective
We aim to:
- Download stock data
- Calculate returns and risk
- Simulate portfolios
- Identify the best one using the **Sharpe Ratio**

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from scipy.optimize import minimize

%matplotlib inline

## 💼 Step 1: Choose Your Stocks

In [31]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']  # You can customize this list
stock_names = ['Apple', 'Microsoft', 'Alphabet (Google)', 'Amazon', 'Meta (Facebook)']

print("Selected Stocks:")
for i, (ticker, name) in enumerate(zip(tickers, stock_names)):
    print(f"{i+1}. {name} ({ticker})")

Selected Stocks:
1. Apple (AAPL)
2. Microsoft (MSFT)
3. Alphabet (Google) (GOOGL)
4. Amazon (AMZN)
5. Meta (Facebook) (META)


## 📥 Step 2: Download Historical Data

In [32]:
data = yf.download(tickers, start='2018-01-01', end='2023-01-01', auto_adjust=False)
data.dropna(inplace=True)
adj_close_data = data['Adj Close']

[*********************100%***********************]  5 of 5 completed


## 🔁 Step 3: Calculate Daily Returns

In [33]:
daily_returns = adj_close_data.pct_change().dropna()

## 📊 Step 4: Calculate Mean Returns and Covariance

In [34]:
mean_returns = daily_returns.mean() * 252
cov_matrix = daily_returns.cov() * 252


## ⚙️ Step 5: Define Portfolio Performance Functions

In [35]:
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.dot(weights, mean_returns)
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return returns, std


In [36]:
def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
    p_return, p_volatility = portfolio_performance(weights, mean_returns, cov_matrix)
    if p_volatility == 0:
        return 1e10  # penalize zero volatility portfolios
    sharpe = (p_return - risk_free_rate) / p_volatility
    return -sharpe

## 🚧 Step 6: Optimization Constraints

In [37]:
num_assets = len(tickers)
init_guess = num_assets * [1. / num_assets]
bounds = tuple((0, 1) for _ in range(num_assets))
constraints = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1}

## 🚀 Step 7: Optimize the Portfolio

In [38]:
opt_result = minimize(
    negative_sharpe_ratio,
    init_guess,
    args=(mean_returns, cov_matrix),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)
if not opt_result.success:
    raise ValueError("Optimization failed: " + opt_result.message)

In [39]:
opt_weights = opt_result.x
opt_return, opt_volatility = portfolio_performance(opt_weights, mean_returns, cov_matrix)
opt_sharpe = (opt_return - 0.01) / opt_volatility



## ✅ Step 8: Show Optimized Portfolio

In [41]:

print("Optimal Portfolio Weights:")
for ticker, weight in zip(tickers, opt_weights):
    print(f"{ticker}: {weight:.2%}")

print(f"\nExpected Annual Return: {opt_return:.2%}")
print(f"Annual Volatility: {opt_volatility:.2%}")
print(f"Sharpe Ratio: {opt_sharpe:.2f}")


Optimal Portfolio Weights:
AAPL: 49.07%
MSFT: 0.00%
GOOGL: 0.00%
AMZN: 0.00%
META: 50.93%

Expected Annual Return: 27.64%
Annual Volatility: 30.35%
Sharpe Ratio: 0.88


## 🧠 Conclusion

You just created an optimized portfolio using **Modern Portfolio Theory**!

You learned to:
- Download and analyze stock data
- Calculate return & risk
- Maximize the Sharpe Ratio 🚀

Try it with different stocks or time ranges for more insights.